# 상관관계

## user_info 랜덤 생성

In [1]:
import pandas as pd
import math
import re
import joblib
import random

# =========================================================
# 1. 데이터 불러오기
# =========================================================
user_df = pd.read_csv(
    "../../data/03_results/clothes/user_info.csv"
)
perfume_classification = pd.read_csv(
    "../../data/03_results/perfume/perfume_classification.csv"
)
perfume_df = pd.read_csv(
    "../../data/03_results/perfume/perfume.csv"
)
perfume_season = pd.read_csv(
    "../../data/03_results/perfume/perfume_season.csv"
)

top_bottom = pd.read_csv(
    "../../data/03_results/clothes/상의_하의.csv"
)
onepiece = pd.read_csv(
    "../../data/03_results/clothes/원피스.csv"
)

clothes_color = pd.read_csv(
    "../../data/03_results/clothes/clothes_color.csv"
)
perfume_color = pd.read_csv(
    "../../data/03_results/perfume/perfume_color.csv"
)

### 향수 Rating Value 타입 및 , 변경

In [2]:
# 향수 평점 데이터 타입 변환
perfume_df["RatingValue"] = (
    perfume_df["RatingValue"]
    .astype(str)
    .str.replace(",", ".", regex=False)
    .astype(float)
)

### user_info.csv 랜덤으로 채우기

In [8]:
user_seasons = ["봄", "여름", "가을", "겨울"]
dislikes = [
    "woody",
    "lavender",
    "animalic",
    "powdery",
]  # 필요시 perfume_df에서 가져와도 됨

# 고유값 리스트 추출
top_ids = top_bottom["식별자"].dropna().astype(int).unique().tolist()
bottom_ids = top_bottom["식별자"].dropna().astype(int).unique().tolist()
onepiece_ids = onepiece["식별자"].dropna().astype(int).unique().tolist()

top_colors = top_bottom["상의_색상"].dropna().unique().tolist()
bottom_colors = top_bottom["하의_색상"].dropna().unique().tolist()
onepiece_colors = onepiece["원피스_색상"].dropna().unique().tolist()

top_categories = top_bottom["상의_카테고리"].dropna().unique().tolist()
bottom_categories = top_bottom["하의_카테고리"].dropna().unique().tolist()
# onepiece_categories = onepiece["원피스_카테고리"].dropna().unique().tolist()

# 사용자 수 지정
N = 10  

data = []

for i in range(1, N + 1):
    if random.random() < 0.4:  # 20%는 원피스 사용자
        data.append(
            {
                "사용자_식별자": i,
                "상의_식별자": None,
                "상의_카테고리": None,
                "상의_색상": None,
                "하의_식별자": None,
                "하의_카테고리": None,
                "하의_색상": None,
                "원피스_식별자": random.choice(onepiece_ids),
                # "원피스_카테고리": random.choice(onepiece_categories),
                "원피스_색상": random.choice(onepiece_colors),
                "계절": random.choice(user_seasons),
                "비선호_향조": random.choice(dislikes),
            }
        )
    else:  # 상·하의 사용자
        data.append(
            {
                "사용자_식별자": i,
                "상의_식별자": random.choice(top_ids),
                "상의_카테고리": random.choice(top_categories),
                "상의_색상": random.choice(top_colors),
                "하의_식별자": random.choice(bottom_ids),
                "하의_카테고리": random.choice(bottom_categories),
                "하의_색상": random.choice(bottom_colors),
                "원피스_식별자": None,
                # "원피스_카테고리": None,
                "원피스_색상": None,
                "계절": random.choice(user_seasons),
                "비선호_향조": random.choice(dislikes),
            }
        )

# 데이터프레임으로 변환
user_df = pd.DataFrame(data)

# 필요한 경우 저장도 가능
# user_df.to_csv("랜덤_user_info.csv", index=False)

print(f" 데이터 기반 랜덤 사용자 생성 완료, 총 {len(user_df)}명")

 데이터 기반 랜덤 사용자 생성 완료, 총 10명


In [9]:
user_df

,사용자_식별자,상의_식별자,상의_카테고리,상의_색상,하의_식별자,하의_카테고리,하의_색상,원피스_식별자,원피스_색상,계절,비선호_향조
0,1,NaN,None,None,NaN,None,None,196902.0,골드,겨울,powdery
1,2,NaN,None,None,NaN,None,None,1077404.0,퍼플,겨울,powdery
2,3,NaN,None,None,NaN,None,None,1006.0,골드,여름,powdery
3,4,1167231.0,브라탑,베이지,1346019.0,래깅스,레드,NaN,None,겨울,powdery
4,5,NaN,None,None,NaN,None,None,8241.0,오렌지,겨울,animalic
5,6,NaN,None,None,NaN,None,None,1072683.0,카키,겨울,lavender
6,7,NaN,None,None,NaN,None,None,2178.0,그레이,여름,powdery
7,8,NaN,None,None,NaN,None,None,570550.0,스카이블루,겨울,powdery
8,9,NaN,None,None,NaN,None,None,117524.0,와인,봄,lavender
9,10,482031.0,브라탑,스카이블루,636767.0,래깅스,실버,NaN,None,여름,powdery


### myscore 계산 및 ratingvale, user_id 포함 데이터프레임 생성

In [10]:
# =========================================================
# 2. 모든 사용자에 대해 반복
# =========================================================
score_list = []

for idx, user_row in user_df.iterrows():
    user_id = user_row["사용자_식별자"]
    
    # 사용자의 비선호_향조 추출
    dislike_raw = user_df.loc[
        user_df["사용자_식별자"] == user_id, 
        "비선호_향조"
    ].iloc[0]

    # 리스트로 변경
    if isinstance(dislike_raw, str):
        dislike_accords = [x.strip() for x in dislike_raw.split(",")]
    elif isinstance(dislike_raw, list):
        dislike_accords = dislike_raw
    else:
        dislike_accords = []
    
    perfume_df_user = perfume_df.copy()
    # 2) perfume에서 제외 조건 만들기
    mask_exclude = perfume_df_user[["mainaccord1", "mainaccord2", "mainaccord3"]].isin(
        dislike_accords
    ).any(axis=1)

    # 3) 비선호 향조가 포함된 향수 제외
    perfume_df_user = perfume_df_user[~mask_exclude].reset_index(drop=True)

    ## A. 스타일&향조
    # =========================================================
    # 3. 스타일 예측
    # =========================================================
    df_1 = user_df.drop(columns=["계절", "비선호_향조"], axis=1)

    def merge_clothes(user_df, clothes_df, clothes_type):
        cols = [
            "식별자",
            f"{clothes_type}_소재",
            f"{clothes_type}_핏",
            f"{clothes_type}_프린트",
            f"{clothes_type}_디테일",
            f"{clothes_type}_넥라인",
            "서브스타일",
        ]
        if clothes_type == "상의":
            cols.append("상의_소매기장")
        else:
            cols.append(f"{clothes_type}_기장")

        cols = [c for c in cols if c in clothes_df.columns]

        user_df = user_df.merge(
            clothes_df[cols],
            left_on=f"{clothes_type}_식별자",
            right_on="식별자",
            how="left",
        ).drop(columns=["식별자"], errors="ignore")

        user_df = user_df.rename(columns={"서브스타일": f"{clothes_type}_서브스타일"})
        return user_df

    df_1 = merge_clothes(df_1, top_bottom, "상의")
    df_1 = merge_clothes(df_1, top_bottom, "하의")
    df_1 = merge_clothes(df_1, onepiece, "원피스")

    df_1["상의_카테고리"] = df_1["상의_카테고리"].replace({"브라탑": "탑"})

    BASE_PATH = "../2_data analysis/clothes/"
    model_0 = joblib.load(BASE_PATH + "0_style_model.pkl")
    encoder_0 = joblib.load(BASE_PATH + "0_clothes_encoder.pkl")
    label_encoder_0 = joblib.load(BASE_PATH + "0_style_label_encoder.pkl")

    model_1 = joblib.load(BASE_PATH + "1_style_model.pkl")
    encoder_1 = joblib.load(BASE_PATH + "1_clothes_encoder.pkl")
    label_encoder_1 = joblib.load(BASE_PATH + "1_style_label_encoder.pkl")

    row = df_1.iloc[-1]

    if pd.isna(row["원피스_식별자"]):
        model, encoder, label_encoder = model_0, encoder_0, label_encoder_0
        row["색상_조합"] = f"{row['상의_색상']}_{row['하의_색상']}"
        row["핏_조합"] = f"{row['상의_핏']}_{row['하의_핏']}"
        train_cols = encoder.feature_names_in_
    else:
        model, encoder, label_encoder = model_1, encoder_1, label_encoder_1
        train_cols = encoder.feature_names_in_

    row_df = pd.DataFrame([row[train_cols]])
    train_cols = [col for col in encoder.feature_names_in_ if col in row.index]

    row_df[train_cols] = encoder.transform(row_df[train_cols].astype("object"))

    user_style = label_encoder.inverse_transform([model.predict(row_df)[0]])[0]
    print(f"선택된 사용자: {user_id}, 예측 스타일: {user_style}")

    # =========================================================
    # 4. 스타일 점수
    # =========================================================
    style_fragrance_score = {
        "로맨틱": {
            "플로럴향, 달콤한향": 46.15,
            "싱그러운 풀 향": 7.69,
            "머스크같은 중후한향": 0.0,
            "파우더느낌의 부드러운향": 30.77,
            "시원하고 신선한 바다 향": 15.38,
            "감귤류의 상큼한 향": 0.0,
            "라벤더같은 상쾌한향": 0.0,
        },
        "섹시": {
            "플로럴향, 달콤한향": 20.0,
            "싱그러운 풀 향": 40.0,
            "머스크같은 중후한향": 40.0,
            "파우더느낌의 부드러운향": 0.0,
            "시원하고 신선한 바다 향": 0.0,
            "감귤류의 상큼한 향": 0.0,
            "라벤더같은 상쾌한향": 0.0,
        },
        "소피스트케이티드": {
            "플로럴향, 달콤한향": 30.0,
            "싱그러운 풀 향": 10.0,
            "머스크같은 중후한향": 10.0,
            "파우더느낌의 부드러운향": 40.0,
            "시원하고 신선한 바다 향": 10.0,
            "감귤류의 상큼한 향": 0.0,
            "라벤더같은 상쾌한향": 0.0,
        },
        "스포티": {
            "플로럴향, 달콤한향": 14.29,
            "싱그러운 풀 향": 9.52,
            "머스크같은 중후한향": 0.0,
            "파우더느낌의 부드러운향": 4.76,
            "시원하고 신선한 바다 향": 57.14,
            "감귤류의 상큼한 향": 14.29,
            "라벤더같은 상쾌한향": 0.0,
        },
        "클래식": {
            "플로럴향, 달콤한향": 9.09,
            "싱그러운 풀 향": 12.12,
            "머스크같은 중후한향": 6.06,
            "파우더느낌의 부드러운향": 21.21,
            "시원하고 신선한 바다 향": 36.36,
            "감귤류의 상큼한 향": 6.06,
            "라벤더같은 상쾌한향": 9.09,
        },
        "젠더리스": {
            "플로럴향, 달콤한향": 21.43,
            "싱그러운 풀 향": 21.43,
            "머스크같은 중후한향": 0.0,
            "파우더느낌의 부드러운향": 28.57,
            "시원하고 신선한 바다 향": 14.29,
            "감귤류의 상큼한 향": 14.29,
            "라벤더같은 상쾌한향": 0.0,
        },
        "아방가르드": {
            "플로럴향, 달콤한향": 11.11,
            "싱그러운 풀 향": 5.56,
            "머스크같은 중후한향": 0.0,
            "파우더느낌의 부드러운향": 16.67,
            "시원하고 신선한 바다 향": 44.44,
            "감귤류의 상큼한 향": 16.67,
            "라벤더같은 상쾌한향": 5.56,
        },
    }

    perfume_classification["style_score"] = perfume_classification["fragrance"].apply(
        lambda x: style_fragrance_score.get(user_style, {}).get(x, 0)
    )

    ## B. 색상
    # =========================================================
    # 5. 색상 점수
    # =========================================================
    def parse_rgb(x):
        nums = re.findall(r"\d+", str(x))
        return tuple(map(int, nums[:3]))

    clothes_color["rgb"] = clothes_color["rgb_tuple"].apply(parse_rgb)
    perfume_color["rgb"] = perfume_color["color"].apply(parse_rgb)

    clothes_color_map = dict(zip(clothes_color["color"], clothes_color["rgb"]))
    perfume_color_map = dict(zip(perfume_color["mainaccord"], perfume_color["rgb"]))

    def calc_color_score(c_vec, f_vec):
        dist = math.sqrt(sum((a - b) ** 2 for a, b in zip(c_vec, f_vec)))
        return 100 * (1 - dist / (255 * math.sqrt(3)))

    def mix_fragrance(a1, a2, a3):
        return [a1[i] * 0.6 + a2[i] * 0.3 + a3[i] * 0.1 for i in range(3)]

    if pd.notna(user_row["원피스_색상"]):
        clothes_vec = clothes_color_map[user_row["원피스_색상"]]
    else:
        top = clothes_color_map[user_row["상의_색상"]]
        bottom = clothes_color_map[user_row["하의_색상"]]
        clothes_vec = [top[i] * 0.7 + bottom[i] * 0.3 for i in range(3)]

    perfume_df_user["color_score"] = perfume_df_user.apply(
        lambda r: calc_color_score(
            clothes_vec,
            mix_fragrance(
                perfume_color_map[r["mainaccord1"]],
                perfume_color_map[r["mainaccord2"]],
                perfume_color_map[r["mainaccord3"]],
            ),
        ),
        axis=1,
    )

    ## C. 계절
    # =========================================================
    # 6. 계절 점수
    # =========================================================
    season_map = {"봄": "spring", "여름": "summer", "가을": "fall", "겨울": "winter"}
    user_season = season_map[user_row["계절"]]

    perfume_season["season_score"] = perfume_season[user_season] / (
        perfume_season[["spring", "summer", "fall", "winter"]].sum(axis=1)
    )
    perfume_season["season_score"] = perfume_season["season_score"].fillna(0) * 100

    # 💥 중복 방지를 위해 'season_score' 미리 삭제
    if "season_score" in perfume_df_user.columns:
        perfume_df_user = perfume_df_user.drop(columns=["season_score"])

    perfume_df_user = perfume_df_user.merge(
        perfume_season[["perfume_id", "season_score"]], on="perfume_id", how="left"
    )

    # 향수별 점수 계산 후, 사용자 ID도 같이 붙이기
    final_df = perfume_df_user[["perfume_id", "color_score", "season_score"]].copy()
    final_df = final_df.merge(
        perfume_classification[["perfume_id", "style_score"]], on="perfume_id", how="left"
    )
    final_df["user_id"] = user_id  # ✅ 사용자 ID 직접 넣기


    final_df["myscore"] = (
        final_df["style_score"] + final_df["color_score"] + final_df["season_score"]
    )

    final_df["RatingValue"] = perfume_df_user["RatingValue"]  # 평점도 같이 붙이기

    score_list.append(final_df)  # 사용자별 향수 점수 전체 append!
    final_result = pd.concat(score_list, ignore_index=True)

C:\Users\Admin\AppData\Local\Temp\ipykernel_4512\2837365471.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row["색상_조합"] = f"{row['상의_색상']}_{row['하의_색상']}"
C:\Users\Admin\AppData\Local\Temp\ipykernel_4512\2837365471.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row["색상_조합"] = f"{row['상의_색상']}_{row['하의_색상']}"
C:\Users\Admin\AppData\Local\Temp\ipykernel_4512\2837365471.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[

선택된 사용자: 1, 예측 스타일: 스포티
선택된 사용자: 2, 예측 스타일: 스포티


C:\Users\Admin\AppData\Local\Temp\ipykernel_4512\2837365471.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row["색상_조합"] = f"{row['상의_색상']}_{row['하의_색상']}"
C:\Users\Admin\AppData\Local\Temp\ipykernel_4512\2837365471.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row["색상_조합"] = f"{row['상의_색상']}_{row['하의_색상']}"
C:\Users\Admin\AppData\Local\Temp\ipykernel_4512\2837365471.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[

선택된 사용자: 3, 예측 스타일: 스포티
선택된 사용자: 4, 예측 스타일: 스포티


C:\Users\Admin\AppData\Local\Temp\ipykernel_4512\2837365471.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row["색상_조합"] = f"{row['상의_색상']}_{row['하의_색상']}"
C:\Users\Admin\AppData\Local\Temp\ipykernel_4512\2837365471.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row["색상_조합"] = f"{row['상의_색상']}_{row['하의_색상']}"
C:\Users\Admin\AppData\Local\Temp\ipykernel_4512\2837365471.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[

선택된 사용자: 5, 예측 스타일: 스포티
선택된 사용자: 6, 예측 스타일: 스포티


C:\Users\Admin\AppData\Local\Temp\ipykernel_4512\2837365471.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row["색상_조합"] = f"{row['상의_색상']}_{row['하의_색상']}"
C:\Users\Admin\AppData\Local\Temp\ipykernel_4512\2837365471.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row["색상_조합"] = f"{row['상의_색상']}_{row['하의_색상']}"
C:\Users\Admin\AppData\Local\Temp\ipykernel_4512\2837365471.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[

선택된 사용자: 7, 예측 스타일: 스포티
선택된 사용자: 8, 예측 스타일: 스포티
선택된 사용자: 9, 예측 스타일: 스포티
선택된 사용자: 10, 예측 스타일: 스포티


C:\Users\Admin\AppData\Local\Temp\ipykernel_4512\2837365471.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row["색상_조합"] = f"{row['상의_색상']}_{row['하의_색상']}"
C:\Users\Admin\AppData\Local\Temp\ipykernel_4512\2837365471.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row["색상_조합"] = f"{row['상의_색상']}_{row['하의_색상']}"
C:\Users\Admin\AppData\Local\Temp\ipykernel_4512\2837365471.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  row[

In [11]:
final_result

,perfume_id,color_score,season_score,style_score,user_id,myscore,RatingValue
0,0,74.240806,16.666667,14.29,1,105.197473,2.89
1,1,55.352813,26.415094,14.29,1,96.057908,3.08
2,3,53.890646,32.075486,14.29,1,100.256132,3.16
3,5,70.639897,6.779647,14.29,1,91.709544,3.22
4,7,63.040837,50.000000,0.00,1,113.040837,3.24
...,...,...,...,...,...,...,...
12987,1467,50.959450,19.298238,0.00,10,70.257688,4.61
12988,1470,45.928773,0.000000,0.00,10,45.928773,4.63
12989,1471,54.191675,11.600935,0.00,10,65.792609,4.64
12990,1473,90.682003,32.258076,14.29,10,137.230079,4.67


In [12]:
final_result[
    ["style_score", "color_score", "season_score", "myscore"]
].corrwith(final_result["RatingValue"])

style_score    -0.141854
color_score    -0.013302
season_score    0.053416
myscore        -0.025469
dtype: float64

In [13]:
user_corr = (
    final_result
    .groupby("user_id")
    .apply(
        lambda x: x[
            ["style_score", "color_score", "season_score", "myscore"]
        ].corrwith(x["RatingValue"])
    )
)

user_corr


C:\Users\Admin\AppData\Local\Temp\ipykernel_4512\1162520001.py:2: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  final_result


,style_score,color_score,season_score,myscore
user_id,,,,
1,-0.140536,0.027141,0.263164,0.164296
2,-0.140536,0.021956,0.263164,0.142547
3,-0.140536,0.027141,-0.228614,-0.189984
4,-0.140536,-0.113415,0.263164,0.057487
5,-0.136410,-0.011636,0.281218,0.142944
6,-0.146333,-0.023313,0.280241,0.118450
7,-0.140536,0.013718,-0.228614,-0.225992
8,-0.140536,-0.067334,0.263164,0.090580
9,-0.146333,0.038620,-0.254293,-0.217635
